## Time Series Forecasting using Gaussian Process Regression

The following analysis were largely borrowed from [this tutorial](https://docs.pymc.io/notebooks/GP-MaunaLoa.html).

Gaussian processes, timeseries analysis, Bayesian modelling

In [1]:
import pymc3 as pm
import pandas as pd
import numpy as np
import theano.tensor as tt

from bokeh.plotting import figure, show
from bokeh.models import BoxAnnotation, Span, Label, Legend
from bokeh.io import output_notebook
from bokeh.palettes import brewer
output_notebook()

WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named 'mkl'


Loading BokehJS ...

### Preprocess

In [2]:
df = pd.read_excel('concat_raw_o2.xlsx')
df[['DATE', 'DAY_OF_WEEK']] = df['DATE'].str.split(' ', n=1, expand=True)
df['DATE'] = pd.to_datetime(df['DATE'])
df = df.sort_values('DATE')
df.set_index('DATE', inplace=True)
df = df['2016-01-01':'2019-08-04']
df = df.loc[df['IND REV'] >=0]
df = df.loc[~df.index.duplicated(keep='first')]
data_columns = ['OCC %', 'REVENUE', 'AVE RATE']
# Resample to monthly frequency, aggregating with mean
monthly_mean = df[data_columns].resample('M').mean()

In [9]:
def dates_to_idx(timelist):
    reference_time = pd.to_datetime('2016-01-31')
    t = (timelist - reference_time) / pd.Timedelta(1, "Y")
    return np.asarray(t)

t = dates_to_idx(monthly_mean.index)

# normalize CO2 levels
y = monthly_mean["REVENUE"].values
first_rev = y[0]
std_rev = np.std(y)
y_n = (y - first_rev) / std_rev

monthly_mean = monthly_mean.assign(t = t)
monthly_mean = monthly_mean.assign(y_n = y_n)

In [10]:
# split into training and test set
sep_idx = monthly_mean.index.searchsorted(pd.to_datetime("2019-01-01"))
data_early = monthly_mean.iloc[:sep_idx+1, :]
data_later = monthly_mean.iloc[sep_idx:, :]

In [14]:
# make plot

p = figure(x_axis_type='datetime', title='Monthly Average Revenue of Ocean Two',
           plot_width=550, plot_height=350)
p.yaxis.axis_label = 'Revenue'
p.xaxis.axis_label = 'Date'
# Predict 2019-01-01 onwards
predict_region = BoxAnnotation(left=pd.to_datetime("2019-01-01"),
                               fill_alpha=0.1, fill_color="firebrick")
p.add_layout(predict_region)
ppm400 = Span(location=400,
              dimension='width', line_color='red',
              line_dash='dashed', line_width=2)
p.add_layout(ppm400)

p.line(monthly_mean.index, monthly_mean['REVENUE'],
       line_width=2, line_color="black", alpha=0.5)
p.circle(monthly_mean.index, monthly_mean['REVENUE'],
         line_color="black", alpha=0.1, size=2)

train_label = Label(x=100, y=30, x_units='screen', y_units='screen',
                    text='Training Set', render_mode='css', border_line_alpha=0.0,
                    background_fill_alpha=0.0)
test_label  = Label(x=510, y=80, x_units='screen', y_units='screen',
                    text='Test Set', render_mode='css', border_line_alpha=0.0,
                    background_fill_alpha=0.0)

p.add_layout(train_label)
p.add_layout(test_label)
show(p)

### The Gaussian Process (GP) model in PyMC3

Below is the actual model. Each of the three component GPs is constructed separately. Since we are doing maximum a-posteriori  (MAP), we use Marginal GPs and lastly call the .marginal_likelihood method to specify the marginal posterior.

In [16]:
# pull out normalized data
t = data_early["t"].values[:,None]
y = data_early["y_n"].values

with pm.Model() as model:
    # yearly periodic component x long term trend
    η_per = pm.HalfCauchy("η_per", beta=2, testval=1.0)
    ℓ_pdecay = pm.Gamma("ℓ_pdecay", alpha=10, beta=0.075)
    period  = pm.Normal("period", mu=1, sigma=0.05)
    ℓ_psmooth = pm.Gamma("ℓ_psmooth ", alpha=4, beta=3)
    cov_seasonal = η_per**2 * pm.gp.cov.Periodic(1, period, ℓ_psmooth) \
                            * pm.gp.cov.Matern52(1, ℓ_pdecay)
    gp_seasonal = pm.gp.Marginal(cov_func=cov_seasonal)

    # small/medium term irregularities
    η_med = pm.HalfCauchy("η_med", beta=0.5, testval=0.1)
    ℓ_med = pm.Gamma("ℓ_med", alpha=2, beta=0.75)
    α = pm.Gamma("α", alpha=5, beta=2)
    cov_medium = η_med**2 * pm.gp.cov.RatQuad(1, ℓ_med, α)
    gp_medium = pm.gp.Marginal(cov_func=cov_medium)

    # long term trend
    η_trend = pm.HalfCauchy("η_trend", beta=2, testval=2.0)
    ℓ_trend = pm.Gamma("ℓ_trend", alpha=4, beta=0.1)
    cov_trend = η_trend**2 * pm.gp.cov.ExpQuad(1, ℓ_trend)
    gp_trend = pm.gp.Marginal(cov_func=cov_trend)

    # noise model
    η_noise = pm.HalfNormal("η_noise", sigma=0.5, testval=0.05)
    ℓ_noise = pm.Gamma("ℓ_noise", alpha=2, beta=4)
    σ  = pm.HalfNormal("σ",  sigma=0.25, testval=0.05)
    cov_noise = η_noise**2 * pm.gp.cov.Matern32(1, ℓ_noise) +\
                pm.gp.cov.WhiteNoise(σ)

    # The Gaussian process is a sum of these three components
    gp = gp_seasonal + gp_medium + gp_trend

    # Since the normal noise model and the GP are conjugates, we use `Marginal` with the `.marginal_likelihood` method
    y_ = gp.marginal_likelihood("y", X=t, y=y, noise=cov_noise)

    # this line calls an optimizer to find the MAP
    mp = pm.find_MAP(include_transformed=True)

/opt/tljh/user/lib/python3.6/site-packages/theano/tensor/basic.py:6611: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result[diagonal_slice] = x
/opt/tljh/user/lib/python3.6/site-packages/pymc3/tuning/starting.py:61: UserWarning: find_MAP should not be used to initialize the NUTS sampler, simply call pymc3.sample() and it will automatically initialize NUTS in a better way.
  warnings.warn('find_MAP should not be used to initialize the NUTS sampler, simply call pymc3.sample() and it will automatically initialize NUTS in a better way.')
/opt/tljh/user/lib/python3.6/site-packages/theano/tensor/basic.py:6611: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be 

In [18]:
# display the results, dont show transformed parameter values
sorted([name+":"+str(mp[name]) for name in mp.keys() if not name.endswith("_")])

['period:1.0085245409766752',
 'α:1.988373307058695',
 'η_med:0.1636945535018351',
 'η_noise:0.00016401151534963817',
 'η_per:1.7700970160225573',
 'η_trend:0.002701133481886187',
 'σ:0.2598854046366373',
 'ℓ_med:1.3671454433337313',
 'ℓ_noise:0.25012334444737233',
 'ℓ_pdecay:120.15043410643221',
 'ℓ_psmooth :0.808197481374807',
 'ℓ_trend:30.004234056607558']

In [28]:
# predict at a 10 day granularity
dates = pd.date_range(start='01/01/2016', end="01/01/2019", freq="10D")
tnew = dates_to_idx(dates)[:,None]

print("Predicting with gp ...")
mu, var = gp.predict(tnew, point=mp, diag=True)
mean_pred = mu*std_rev + first_rev
var_pred  = var*std_rev**2

# make dataframe to store fit results
fit = pd.DataFrame({"t": tnew.flatten(),
                    "mu_total": mean_pred,
                    "sd_total": np.sqrt(var_pred)},
                   index=dates)

print("Predicting with gp_trend ...")
mu, var = gp_trend.predict(tnew, point=mp,
                           given={"gp": gp, "X": t, "y": y, "noise": cov_noise},
                           diag=True)
fit = fit.assign(mu_trend = mu*std_rev + first_rev,
                 sd_trend = np.sqrt(var*std_rev**2))

print("Predicting with gp_medium ...")
mu, var = gp_medium.predict(tnew, point=mp,
                            given={"gp": gp, "X": t, "y": y, "noise": cov_noise},
                            diag=True)
fit = fit.assign(mu_medium = mu*std_rev + first_rev,
                 sd_medium = np.sqrt(var*std_rev**2))

print("Predicting with gp_seasonal ...")
mu, var = gp_seasonal.predict(tnew, point=mp,
                              given={"gp": gp, "X": t, "y": y, "noise": cov_noise},
                              diag=True)
fit = fit.assign(mu_seasonal = mu*std_rev + first_rev,
                 sd_seasonal = np.sqrt(var*std_rev**2))
print("Done")

Predicting with gp ...
Predicting with gp_trend ...
Predicting with gp_medium ...
Predicting with gp_seasonal ...
Done


In [29]:
## plot the components
p = figure(title="Decomposition of the Revenue",
           x_axis_type='datetime', plot_width=750, plot_height=550)
p.yaxis.axis_label = 'Revenue'
p.xaxis.axis_label = 'Date'

# plot mean and 2σ region of total prediction
upper = fit.mu_total + 2*fit.sd_total
lower = fit.mu_total - 2*fit.sd_total
band_x = np.append(fit.index.values, fit.index.values[::-1])
band_y = np.append(lower, upper[::-1])

# total fit
p.line(fit.index, fit.mu_total,
       line_width=1, line_color="firebrick", legend="Total fit")
p.patch(band_x, band_y,
        color="firebrick", alpha=0.6, line_color="white")

# trend
p.line(fit.index, fit.mu_trend,
       line_width=1, line_color="blue", legend="Long term trend")

# medium
p.line(fit.index, fit.mu_medium,
       line_width=1, line_color="green", legend="Medium range variation")

# seasonal
p.line(fit.index, fit.mu_seasonal,
       line_width=1, line_color="orange", legend="Seasonal process")

# true value
p.circle(data_early.index, data_early['REVENUE'],
         color="black", legend="Observed data")
p.legend.location = "bottom_left"
show(p)

In [33]:
# plot several years 

p = figure(title="Several years of the seasonal component",
           plot_width=550, plot_height=350)
p.yaxis.axis_label = 'Δ Revenue'
p.xaxis.axis_label = 'Month'

colors = brewer['Paired'][5]
years = ["2019", "2020", "2021", "2023", "2025"]

for i, year in enumerate(years):
    dates = pd.date_range(start="1/1/"+year, end="12/31/"+year, freq="10D")
    tnew = dates_to_idx(dates)[:,None]

    print("Predicting year", year)
    mu, var = gp_seasonal.predict(tnew, point=mp, diag=True,
                                  given={"gp": gp, "X": t, "y": y, "noise": cov_noise})
    mu_pred = mu*std_rev

    # plot mean
    x = np.asarray((dates - dates[0])/pd.Timedelta(1, "M")) + 1
    p.line(x, mu_pred,
           line_width=1, line_color=colors[i], legend=year)

p.legend.location = "bottom_left"
show(p)

Predicting year 2019
Predicting year 2020
Predicting year 2021
Predicting year 2023
Predicting year 2025


## Forecast into future years

In [31]:
dates = pd.date_range(start="01/01/2016", end="12/31/2021", freq="10D")
tnew = dates_to_idx(dates)[:,None]

print("Sampling gp predictions ...")
mu_pred, cov_pred = gp.predict(tnew, point=mp)

# draw samples, and rescale
n_samples = 2000
samples = pm.MvNormal.dist(mu=mu_pred, cov=cov_pred).random(size=n_samples)
samples = samples * std_rev + first_rev

Sampling gp predictions ...


In [26]:
data_total = pd.concat([data_early, data_later], axis=0)

In [32]:
### make plot
p = figure(x_axis_type='datetime', plot_width=700, plot_height=400)
p.yaxis.axis_label = 'Revenue'
p.xaxis.axis_label = 'Date'

### plot mean and 2σ region of total prediction
# scale mean and var
mu_pred_sc = mu_pred * std_rev + first_rev
sd_pred_sc = np.sqrt(np.diag(cov_pred) * std_rev**2 )

upper = mu_pred_sc + 2*sd_pred_sc
lower = mu_pred_sc - 2*sd_pred_sc
band_x = np.append(dates, dates[::-1])
band_y = np.append(lower, upper[::-1])

p.line(dates, mu_pred_sc,
       line_width=2, line_color="firebrick", legend="Total fit")
p.patch(band_x, band_y,
        color="firebrick", alpha=0.6, line_color="white")

# some predictions
idx = np.random.randint(0, samples.shape[0], 10)
p.multi_line([dates]*len(idx), [samples[i,:] for i in idx],
             color="firebrick", alpha=0.5, line_width=0.5)
# true value
#p.line(data_later.index, data_later['CO2'],
#       line_width=2, line_color="black", legend="Observed data")
p.circle(data_total.index, data_total['REVENUE'],
         color="black", legend="Observed data")

ppm400 = Span(location=400,
              dimension='width', line_color='black',
              line_dash='dashed', line_width=1)
p.add_layout(ppm400)
p.legend.location = "bottom_right"
show(p)